In [1]:
from IPython.display import Markdown

### Init

In [2]:
from GPTAssistant import GPTAssistant 
from dotenv import load_dotenv
import os
from datetime import datetime
import pytz

load_dotenv()

OPENAI_KEY = os.getenv("OPENAI_KEY")
kst = pytz.timezone('Asia/Seoul')

In [13]:
today = datetime.now().astimezone(kst)
str_tdy = today.strftime("%A, %B %d, %Y")
str_time = today.strftime("%H:%M")


tools = [
    {
        "type": "function",
        "function": {
            "name": "reserve_meetingroom",
            "description": "Based on the list of entered conversations, identify the meeting location and time.",
            "parameters": {
                "type": "object",
                "properties": {
                    "meeting_room": {
                        "type": "string",
                        "description": "Meeting location inferred from the conversation list.",
                        "enum": ["회의실1", "회의실2", "회의실3"],
                    },
                    "meeting_date": {
                        "type": "string",
                        "description": f"Meeting date inferred from the conversation list and display it in YYYY-mm-dd format. Today’s date is {str_tdy}. If a relative date reference (e.g., '30분 뒤', '내일', '다음 주') is mentioned, calculate the exact date accordingly. If a specific day is inferred but is earlier than today’s date, assume it refers to the same day in the next month. Ensure the date is valid by considering the last day of each month (e.g., January 31, February 28/29, March 31, etc.).",
                    },
                    "meeting_time": {
                        "type": "string",
                        "description": f"Meeting time inferred from the conversation list, displayed in 24-hour format (e.g., 17:30). The current time is {str_time}. If the time reference is ambiguous (e.g., no AM/PM indication), infer it as AM from 09:00 to 11:59 and as PM otherwise. If a relative time (e.g., '30분 뒤', '1시간 뒤') is mentioned, calculate the exact time accordingly. If no specific time is mentioned and no relative time reference exists, return 09:00.",
                    },
                    "meeting_topic": {
                        "type": "string",
                        "description": "Meeting topic inferred from the conversation list: within 30 characters, in 한국어. e.g., 'OO 사업 기획 관련 회의', 'OO사 미팅', etc.",
                    },
                },
                "required": [
                    "meeting_room",
                    "meeting_date",
                    "meeting_time",
                    "meeting_topic",
                ],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [4]:
concept = "You are a auto meetingroom reservation bot. Based on the list of entered conversations, identify the meeting location and time."

In [14]:
assistant = GPTAssistant(
  api_key=OPENAI_KEY,
  model_name="gpt-4o-mini",
  instructions=concept,
  tools=tools,
  return_args_only=True
)

---
### Run

In [15]:
conv = [
  {
    "speaker": "김지영 대리",
    "text":"대표님이 30분 뒤에 우리 팀 다같이 보자네요"
   },
   {
     "speaker": "권은서 주임",
     "text": "헐 무섭다 회의실 예약할까요?"
   },
   {
     "speaker": "권은찬 주임",
     "text": "왜 부르시는지는 모르죠?"
   },
   {
    "speaker": "김지영 대리",
    "text":"네 이유는 모르겠네요;; 일단 회의실 아무데나 잡아주세요"
   },
]

In [16]:
res = assistant.first_chat(str(conv))

In [17]:
Markdown(res["results"][0]['output'])

{'meeting_room': '회의실1', 'meeting_date': '2025-02-27', 'meeting_time': '20:01', 'meeting_topic': '대표님 미팅'}